In [9]:
%matplotlib inline
import pandas as pd
from scipy.stats import pearsonr 
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
import numpy as np

lang2mono = {'EN': 'bert-base-uncased',
             'ID': 'indobert-base-uncased',
             'FR': 'camembert-base',
             'TR': 'bert-base-turkish-uncased',
             'ZH': 'bert-base-chinese',
             'RU': 'rubert-base-cased',
             'DE': 'bert-base-german-dbmdz-uncased',
             'ES': 'bert-base-spanish-wwm-uncased'}

LANG = 'RU'
pretrained= lang2mono[LANG] 

path_human_focus = f'mturk/annotation_result/{LANG}/human_focus_final.csv'
path_human_coverage = f'mturk/annotation_result/{LANG}/human_coverage_final.csv'
human_foc = pd.read_csv(path_human_focus)
human_cov = pd.read_csv(path_human_coverage)

human_foc_PG = human_foc[human_foc['model']=='PG']
human_foc_BERT = human_foc[human_foc['model']=='BERT']
human_cov_PG = human_cov[human_cov['model']=='PG']
human_cov_BERT = human_cov[human_cov['model']=='BERT']

In [10]:
def read_array(arr):
    arr = arr.replace('[','').replace(']','')
    num = arr.split(', ')
    num = [float(a) for a in num]
    return num

def read_array2(arr):
    arr = arr.replace('[','').replace(']','')
    num = arr.split(', ')
    num = [a.replace('\'','') for a in num]
    return num

def match(keys, values):
    res = {}
    for idx, key in enumerate(keys):
        res[key] = values[idx]
    return res
        
def correlation(human_metric, id2score):
    filtered_human = []
    filtered_metric = []
    
    for key in id2score.keys():
        if len(human_metric[human_metric['id']==int(key)]) == 0:
            continue
        human_score = human_metric[human_metric['id'] == int(key)]['score'].tolist()[0]
        filtered_human.append(human_score)
        filtered_metric.append(id2score[key])
    return pearsonr(filtered_human, filtered_metric)[0], len(filtered_human)

def get_max(arr):
    max_id = np.argmax(arr)
    print(max_id+1, '--', arr[max_id])
    min_id = np.argmin(arr)
    print(min_id+1, '--', arr[min_id])
    
def get_best_rank(a,b):
    r1 = [sorted(a).index(x) for x in a]
    r2 = [sorted(b).index(x) for x in b]
    score = 0
    rank = -1
    for i in range(len(r1)):
        tmp = (a[i] + b[i])/2
        if tmp > score:
            rank = i
            score = tmp
    print('Best layer:', rank+1)
    return rank

In [11]:
def read(model, human_score, prec_or_rec):
    path = f'bert_score/{LANG}--{model}--{pretrained}.csv'
    df = pd.read_csv(path)
    xlabel = []; cors = []
    for idx, row in df.iterrows():
        cur_layer = row['layer']
        score = read_array(row[prec_or_rec])
        doc_id = read_array2(row['fnames'])
        id2score = match(doc_id, score)
        
        xlabel.append(cur_layer)
        cors.append(correlation(human_score, id2score)[0])
    return xlabel, cors

In [12]:
# Focus
print ('FOCUS for language:', LANG, 'with model', pretrained)

xlabel_pg, cors_pg = read('PG', human_foc_PG, 'precision')
xlabel_bert, cors_bert = read('BERT', human_foc_BERT, 'precision')
layer = get_best_rank(cors_bert, cors_pg)

print('PG:', cors_pg[layer])
print('BERT:', cors_bert[layer])

FOCUS for language: RU with model rubert-base-cased
Best layer: 4
PG: 0.42013286694278185
BERT: 0.7052891181742704


In [13]:
# Coverage
print ('COVERAGE for language:', LANG,  'with model', pretrained)

xlabel_pg, cors_pg = read('PG', human_cov_PG, 'recall')
xlabel_bert, cors_bert = read('BERT', human_cov_BERT, 'recall')
layer = get_best_rank(cors_bert, cors_pg)

print('PG:', cors_pg[layer])
print('BERT:', cors_bert[layer])

COVERAGE for language: RU with model rubert-base-cased
Best layer: 12
PG: 0.7222650638839587
BERT: 0.6327758709514879
